In [7]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service

service = Service("msedgedriver.exe")  # Chemin vers EdgeDriver
options = webdriver.EdgeOptions()

driver = webdriver.Edge(service=service, options=options)
driver.get("https://fbref.com/en/squads/53a2f082/Real-Madrid-Stats")

print("Edge fonctionne avec Selenium !")
driver.quit()


Edge fonctionne avec Selenium !


In [5]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from bs4 import BeautifulSoup, Comment
import pandas as pd
import time

# Configuration Edge
edge_options = Options()
edge_options.add_argument("--headless")  # Exécution en arrière-plan
edge_options.add_argument("--disable-gpu")
edge_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.59")

# Initialisation du WebDriver Edge
service = Service("msedgedriver.exe")  # Assurez-vous que le fichier est dans le dossier
driver = webdriver.Edge(service=service, options=edge_options)

# URL cible
url = "https://fbref.com/en/squads/53a2f082/2023-2024/Real-Madrid-Stats"

try:
    print("🔄 Chargement de la page...")
    driver.get(url)
    time.sleep(5)  # Attente pour le chargement JS

    # Récupération du HTML
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # Détection des tableaux (visibles + commentaires)
    tables = []
    
    # 1. Tableaux visibles
    visible_tables = soup.find_all('table')
    tables.extend(visible_tables)
    
    # 2. Tableaux dans les commentaires HTML
    comments = soup.find_all(string=lambda text: isinstance(text, Comment))
    for comment in comments:
        if '<table' in comment:
            comment_soup = BeautifulSoup(comment, 'html.parser')
            tables.extend(comment_soup.find_all('table'))

    # Filtrage des tableaux valides
    valid_tables = []
    for table in tables:
        if table.has_attr('id'):
            valid_tables.append(table)
            print(f"📋 Tableau détecté : {table['id']}")

    if not valid_tables:
        print("❌ Aucun tableau valide trouvé")
        exit()

    # Extraction des données
    data = {}
    for table in valid_tables:
        try:
            df = pd.read_html(str(table))[0]
            
            # Nettoyage des colonnes multi-niveaux
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = ['_'.join(col).strip() for col in df.columns.values]
            
            data[table['id']] = df
            print(f"✅ {table['id']} extrait avec succès")
        except Exception as e:
            print(f"⚠️ Erreur sur {table['id']} : {str(e)}")

    # Sauvegarde Excel
    if data:
        with pd.ExcelWriter("real_madrid_stats.xlsx") as writer:
            for table_id, df in data.items():
                df.to_excel(writer, sheet_name=table_id[:31])  # Excel limite à 31 caractères
        print("\n💾 Fichier Excel généré : real_madrid_stats.xlsx")

        # Analyse rapide
        if 'stats_standard' in data:
            df = data['stats_standard']
            print("\n🔍 Top 5 Buteurs:")
            print(df[['Player', 'Gls', 'Ast']].sort_values('Gls', ascending=False).head(5))

finally:
    driver.quit()
    print("🚪 Navigateur fermé")

🔄 Chargement de la page...
📋 Tableau détecté : stats_standard_12
📋 Tableau détecté : matchlogs_for
📋 Tableau détecté : stats_keeper_12
📋 Tableau détecté : stats_keeper_adv_12
📋 Tableau détecté : stats_shooting_12
📋 Tableau détecté : stats_passing_12
📋 Tableau détecté : stats_passing_types_12
📋 Tableau détecté : stats_gca_12
📋 Tableau détecté : stats_defense_12
📋 Tableau détecté : stats_possession_12
📋 Tableau détecté : stats_playing_time_12
📋 Tableau détecté : stats_misc_12
📋 Tableau détecté : results2023-2024121_overall
📋 Tableau détecté : results2023-2024121_home_away
✅ stats_standard_12 extrait avec succès
✅ matchlogs_for extrait avec succès
✅ stats_keeper_12 extrait avec succès
✅ stats_keeper_adv_12 extrait avec succès
✅ stats_shooting_12 extrait avec succès
✅ stats_passing_12 extrait avec succès


C:\Users\pierr\AppData\Local\Temp\ipykernel_29648\1797056197.py:59: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\pierr\AppData\Local\Temp\ipykernel_29648\1797056197.py:59: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\pierr\AppData\Local\Temp\ipykernel_29648\1797056197.py:59: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\pierr\AppData\Local\Temp\ipykernel_29648\1797056197.py:59: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

✅ stats_passing_types_12 extrait avec succès
✅ stats_gca_12 extrait avec succès
✅ stats_defense_12 extrait avec succès
✅ stats_possession_12 extrait avec succès
✅ stats_playing_time_12 extrait avec succès
✅ stats_misc_12 extrait avec succès
✅ results2023-2024121_overall extrait avec succès
✅ results2023-2024121_home_away extrait avec succès


C:\Users\pierr\AppData\Local\Temp\ipykernel_29648\1797056197.py:59: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\pierr\AppData\Local\Temp\ipykernel_29648\1797056197.py:59: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\pierr\AppData\Local\Temp\ipykernel_29648\1797056197.py:59: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\pierr\AppData\Local\Temp\ipykernel_29648\1797056197.py:59: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr


💾 Fichier Excel généré : real_madrid_stats.xlsx
🚪 Navigateur fermé


In [4]:
# Configuration
input_file = "real_madrid_data/statistiques.xlsx"
output_file = "real_madrid_clean.xlsx"

# 1. Charger toutes les feuilles
print("Chargement des données...")
all_sheets = pd.read_excel(input_file, sheet_name=None)

# 2. Fonction de nettoyage améliorée
def clean_sheet(df):
    # Supprimer les lignes d'en-tête répétées et les lignes vides
    df = df[~df.iloc[:, 0].astype(str).str.contains('Rk|Player|Squad|Opponent', na=False)]
    df = df.dropna(how='all')
    
    # Nettoyer les noms de colonnes multi-niveaux
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [f"{col[0]}_{col[1]}".strip('_') if col[1] else col[0] 
                     for col in df.columns.values]
    
    # Supprimer les colonnes entièrement vides
    df = df.dropna(axis=1, how='all')
    
    # Standardiser le nom du joueur et filtrer les lignes vides
    player_col = next((col for col in df.columns if 'Player' in str(col)), None)
    if player_col:
        df = df[df[player_col].notna()]
        df[player_col] = df[player_col].astype(str).str.strip()
        df = df.rename(columns={player_col: 'Player'})
    
    # Convertir les colonnes numériques de manière sécurisée
    for col in df.columns:
        if col not in ['Player', 'Nation', 'Pos', 'Age', 'Matches']:
            try:
                df[col] = pd.to_numeric(df[col])
            except:
                # Si la conversion échoue, on garde la valeur originale
                pass
    
    return df

# 3. Nettoyer chaque feuille
print("🧹 Nettoyage des feuilles...")
clean_data = {}
for sheet_name, df in all_sheets.items():
    try:
        print(f"  - Nettoyage de {sheet_name}...")
        clean_data[sheet_name] = clean_sheet(df)
    except Exception as e:
        print(f"Erreur sur {sheet_name}: {str(e)}")

# 4. Créer une table de référence des joueurs
print("👥 Création de la table des joueurs...")
players = pd.DataFrame()

# Essayer de trouver les colonnes Player, Pos, Age dans différentes feuilles
for sheet_name in ['stats_standard_12', 'stats_playing_time_12', 'stats_misc_12']:
    if sheet_name in clean_data:
        df = clean_data[sheet_name]
        cols = {}
        for col in ['Player', 'Pos', 'Age']:
            matching_cols = [c for c in df.columns if col in str(c)]
            if matching_cols:
                cols[col] = matching_cols[0]
        
        if len(cols) == 3:  # On a trouvé les trois colonnes
            players = df[list(cols.values())].rename(columns=cols)
            players = players.drop_duplicates()
            break

if players.empty:
    # Fallback si on n'a pas trouvé les colonnes standard
    print("Impossible de trouver les colonnes standard, création alternative...")
    for sheet_name, df in clean_data.items():
        if 'Player' in df.columns:
            player_cols = {'Player': 'Player'}
            if 'Pos' in df.columns:
                player_cols['Pos'] = 'Pos'
            if 'Age' in df.columns:
                player_cols['Age'] = 'Age'
            
            temp_df = df[list(player_cols.keys())].rename(columns=player_cols)
            players = pd.concat([players, temp_df])
    
    players = players.drop_duplicates()

# 5. Sauvegarde pour Power BI
print("Sauvegarde pour Power BI...")
with pd.ExcelWriter(output_file) as writer:
    # Sauvegarder chaque feuille nettoyée
    for sheet_name, df in clean_data.items():
        df.to_excel(writer, sheet_name=sheet_name[:31], index=False)
    
    # Ajouter la table des joueurs si elle n'est pas vide
    if not players.empty:
        players.to_excel(writer, sheet_name='Players', index=False)
    
    # Créer une feuille résumé si possible
    try:
        summary_data = []
        for sheet_name, df in clean_data.items():
            if 'Player' in df.columns:
                summary_data.append(df.set_index('Player'))
        
        if summary_data:
            summary_df = pd.concat(summary_data, axis=1)
            # Supprimer les colonnes dupliquées (comme Pos, Age qui peuvent apparaître dans plusieurs feuilles)
            summary_df = summary_df.loc[:, ~summary_df.columns.duplicated()]
            summary_df.reset_index(inplace=True)
            summary_df.to_excel(writer, sheet_name='Summary', index=False)
    except Exception as e:
        print(f"Impossible de créer le résumé: {str(e)}")

print(f"""
✅ Nettoyage terminé !
Fichier sauvegardé : {output_file}

Conseils pour Power BI :
1. Importer le fichier Excel
2. Créer des relations entre les tables via la colonne 'Player'
3. Commencer par analyser les indicateurs clés :
   - Buts (Gls), passes décisives (Ast)
   - Tirs (Sh), tirs cadrés (SoT)
   - Actions défensives (Tkl, Int)
   - Performances des gardiens (Saves, CS)
""")

📂 Chargement des données...
🧹 Nettoyage des feuilles...
  - Nettoyage de stats_standard_12...
  - Nettoyage de stats_keeper_12...
  - Nettoyage de stats_keeper_adv_12...
  - Nettoyage de stats_shooting_12...
  - Nettoyage de stats_passing_12...
  - Nettoyage de stats_passing_types_12...
  - Nettoyage de stats_gca_12...
  - Nettoyage de stats_defense_12...
  - Nettoyage de stats_possession_12...
  - Nettoyage de stats_playing_time_12...
  - Nettoyage de stats_misc_12...
👥 Création de la table des joueurs...
💾 Sauvegarde pour Power BI...
⚠️ Impossible de créer le résumé: Reindexing only valid with uniquely valued Index objects

✅ Nettoyage terminé !
Fichier sauvegardé : real_madrid_clean.xlsx

Conseils pour Power BI :
1. Importer le fichier Excel
2. Créer des relations entre les tables via la colonne 'Player'
3. Commencer par analyser les indicateurs clés :
   - Buts (Gls), passes décisives (Ast)
   - Tirs (Sh), tirs cadrés (SoT)
   - Actions défensives (Tkl, Int)
   - Performances des g

In [11]:
# Charger le fichier et afficher les noms des feuilles
xlsx_file = 'real_madrid_clean.xlsx'  # Assure-toi du bon chemin
sheets = pd.ExcelFile(xlsx_file).sheet_names
print(sheets)

['stats_standard_12', 'stats_keeper_12', 'stats_keeper_adv_12', 'stats_shooting_12', 'stats_passing_12', 'stats_passing_types_12', 'stats_gca_12', 'stats_defense_12', 'stats_possession_12', 'stats_playing_time_12', 'stats_misc_12', 'Players']


In [12]:
def transform_excel_data(file_path):
    # Lire le fichier Excel
    xls = pd.ExcelFile(file_path)
    
    # Créer un dictionnaire pour stocker les DataFrames transformés
    transformed_sheets = {}
    
    # Parcourir chaque feuille
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        
        # Parcourir chaque colonne
        for col in df.columns:
            # Vérifier si la colonne contient des nombres avec des points ou virgules
            if df[col].dtype == 'object':
                try:
                    # Essayer de convertir en float (remplace les virgules par des points)
                    temp_series = df[col].astype(str).str.replace(',', '.').astype(float)
                    
                    # Vérifier si les valeurs converties sont des entiers
                    if all(temp_series.dropna().apply(lambda x: x.is_integer())):
                        # Convertir en int si ce sont des entiers
                        df[col] = temp_series.astype('Int64')  # Int64 permet les NaN
                    else:
                        # Garder en float sinon
                        df[col] = temp_series
                except (ValueError, AttributeError):
                    # Si la conversion échoue, garder la colonne comme object
                    pass
        
        # Stocker le DataFrame transformé
        transformed_sheets[sheet_name] = df
    
    # Écrire les données transformées dans un nouveau fichier Excel
    output_path = file_path.replace('.xlsx', '_transformed.xlsx')
    with pd.ExcelWriter(output_path) as writer:
        for sheet_name, df in transformed_sheets.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    
    return output_path

# Utilisation avec votre fichier
file_path = 'real_madrid_clean.xlsx'
try:
    transformed_file = transform_excel_data(file_path)
    print(f"Transformation réussie ! Fichier enregistré sous : {transformed_file}")
except FileNotFoundError:
    print("Erreur : Le fichier 'real_madrid_clean.xlsx' n'a pas été trouvé dans le dossier du projet.")
except Exception as e:
    print(f"Une erreur s'est produite : {str(e)}")

Transformation réussie ! Fichier enregistré sous : real_madrid_clean_transformed.xlsx


In [13]:
def analyze_column_formats(file_path):
    # Lire le fichier Excel
    xls = pd.ExcelFile(file_path)
    
    print(f"\nAnalyse des formats de colonnes pour le fichier: {file_path}\n")
    print("="*60)
    
    # Parcourir chaque feuille
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        
        print(f"\nFeuille: {sheet_name}")
        print("-"*40)
        
        # Analyser chaque colonne
        for col in df.columns:
            col_type = df[col].dtype
            sample_value = df[col].iloc[0] if len(df[col]) > 0 else None
            
            # Détection plus précise du format
            if col_type == 'object':
                try:
                    # Test de conversion en float
                    temp = pd.to_numeric(df[col].astype(str).str.replace(',', '.'), errors='raise')
                    if all(temp.dropna().apply(lambda x: x.is_integer())):
                        detected_type = "int (détecté dans texte)"
                    else:
                        detected_type = "float (détecté dans texte)"
                except:
                    if df[col].astype(str).str.match(r'^\d{2}/\d{2}/\d{4}$').any():
                        detected_type = "date (détecté dans texte)"
                    else:
                        detected_type = "texte"
            else:
                if pd.api.types.is_integer_dtype(col_type):
                    detected_type = "int"
                elif pd.api.types.is_float_dtype(col_type):
                    detected_type = "float"
                elif pd.api.types.is_datetime64_any_dtype(col_type):
                    detected_type = "datetime"
                else:
                    detected_type = str(col_type)
            
            # Afficher les informations
            print(f"Colonne: {col:20} | Type pandas: {str(col_type):10} | Format détecté: {detected_type:20} | Exemple: {str(sample_value)}")

# Utilisation avec votre fichier
file_path = 'real_madrid_clean.xlsx'
try:
    analyze_column_formats(file_path)
except FileNotFoundError:
    print(f"Erreur: Le fichier {file_path} n'a pas été trouvé dans le dossier courant.")
except Exception as e:
    print(f"Une erreur s'est produite: {str(e)}")


Analyse des formats de colonnes pour le fichier: real_madrid_clean.xlsx


Feuille: stats_standard_12
----------------------------------------
Colonne: joueur               | Type pandas: object     | Format détecté: texte                | Exemple: Federico Valverde
Colonne: pays                 | Type pandas: object     | Format détecté: texte                | Exemple: uy URU
Colonne: position             | Type pandas: object     | Format détecté: texte                | Exemple: MF
Colonne: age                  | Type pandas: int64      | Format détecté: int                  | Exemple: 25
Colonne: MP                   | Type pandas: int64      | Format détecté: int                  | Exemple: 37
Colonne: match_debute         | Type pandas: int64      | Format détecté: int                  | Exemple: 33
Colonne: nombre_de_minute_joue | Type pandas: float64    | Format détecté: float                | Exemple: 2904.0
Colonne: temps_joue_90s       | Type pandas: float64    | Format détec

In [14]:
def remove_empty_columns(file_path):
    # Lire le fichier Excel
    xls = pd.ExcelFile(file_path)
    
    # Créer un dictionnaire pour stocker les DataFrames modifiés
    cleaned_sheets = {}
    
    # Parcourir chaque feuille
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        
        # Supprimer les colonnes où toutes les valeurs sont NaN/None/vides
        before_cols = len(df.columns)
        df.dropna(axis='columns', how='all', inplace=True)
        after_cols = len(df.columns)
        removed_cols = before_cols - after_cols
        
        print(f"Feuille '{sheet_name}': {removed_cols} colonne(s) vide(s) supprimée(s)")
        
        # Stocker le DataFrame modifié
        cleaned_sheets[sheet_name] = df
    
    # Écrire les données nettoyées dans un nouveau fichier Excel
    output_path = file_path.replace('.xlsx', '_no_empty_columns.xlsx')
    with pd.ExcelWriter(output_path) as writer:
        for sheet_name, df in cleaned_sheets.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    
    return output_path

# Utilisation avec votre fichier
file_path = 'real_madrid_clean.xlsx'
try:
    cleaned_file = remove_empty_columns(file_path)
    print(f"\nNettoyage terminé ! Fichier sauvegardé sous : {cleaned_file}")
except FileNotFoundError:
    print(f"Erreur: Le fichier {file_path} n'a pas été trouvé.")
except Exception as e:
    print(f"Une erreur s'est produite : {str(e)}")

Feuille 'stats_standard_12': 16350 colonne(s) vide(s) supprimée(s)
Feuille 'stats_keeper_12': 0 colonne(s) vide(s) supprimée(s)
Feuille 'stats_keeper_adv_12': 0 colonne(s) vide(s) supprimée(s)
Feuille 'stats_shooting_12': 0 colonne(s) vide(s) supprimée(s)
Feuille 'stats_passing_12': 0 colonne(s) vide(s) supprimée(s)
Feuille 'stats_passing_types_12': 0 colonne(s) vide(s) supprimée(s)
Feuille 'stats_gca_12': 0 colonne(s) vide(s) supprimée(s)
Feuille 'stats_defense_12': 0 colonne(s) vide(s) supprimée(s)
Feuille 'stats_possession_12': 0 colonne(s) vide(s) supprimée(s)
Feuille 'stats_playing_time_12': 0 colonne(s) vide(s) supprimée(s)
Feuille 'stats_misc_12': 0 colonne(s) vide(s) supprimée(s)
Feuille 'Players': 0 colonne(s) vide(s) supprimée(s)

Nettoyage terminé ! Fichier sauvegardé sous : real_madrid_clean_no_empty_columns.xlsx
